In [1]:
import pandas as pd  
import numpy as np
import scipy.optimize as sco
import statsmodels.api as sm

data = pd.read_excel('Commodities_py_data.xlsx', sheet_name='Data')
rfr_data = pd.read_excel('Commodities_py_data.xlsx', sheet_name='Rfr_data')
benchmark_data = pd.read_excel('Commodities_py_data.xlsx', sheet_name='Benchmark')
fed_data = pd.read_excel('Commodities_py_data.xlsx', sheet_name='Fed_rate')
inf_data = pd.read_excel('Commodities_py_data.xlsx', sheet_name='Inflation_index')
usd_data = pd.read_excel('Commodities_py_data.xlsx', sheet_name='USD_index')
returns, rfr, benchmark = data.set_index('Date'), rfr_data.set_index('Date'), benchmark_data.set_index('Date')
fed_rate, infl_rate, usd_index = fed_data.set_index('Date'), inf_data.set_index('Date'), usd_data.set_index('Date')

mean_returns = returns.mean()
cov_matrix = returns.cov()
n = len(returns.columns)
simulations = 25000
eq_portfolio_ret = pd.DataFrame(returns.mean(axis=1))
eq_portfolio_ret.columns = ['Eq-weight Portfolio']
risk_free_rate = (rfr.mean()/12)


def single_model():
    X = benchmark
    y = eq_portfolio_ret
    X = sm.add_constant(X)
    
    model = sm.OLS(y, X).fit()
    
    beta = model.params[1]
    display (model.summary())
    return beta

def multi_model():
    multifactors = pd.concat([fed_rate, infl_rate, usd_index, rfr], axis=1)
    X = multifactors
    y = eq_portfolio_ret
    X = sm.add_constant(X)
    
    model2 = sm.OLS(y, X).fit()
    
    return display (model2.summary())

print ("-"*80)
beta = single_model()
print ("-"*80)
multi_model()
print ("-"*80)

def portfolio_performance(weights, mean_returns, cov_matrix):
    returns = np.sum(mean_returns*weights )
    std = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
    return std, returns

def random_portfolios(simulations, mean_returns, cov_matrix, risk_free_rate):
    results = np.zeros((3,simulations))
    weights_record = []
    for i in range(simulations):
        weights = np.random.random(n)
        weights /= np.sum(weights)
        weights_record.append(weights)
        portfolio_std_dev, portfolio_return = portfolio_performance(weights, mean_returns, cov_matrix)
        results[0,i] = portfolio_std_dev
        results[1,i] = portfolio_return
        results[2,i] = (portfolio_return - risk_free_rate) / portfolio_std_dev
    return results, weights_record

def neg_sharpe_ratio(weights, mean_returns, cov_matrix, risk_free_rate):
    p_vol, p_ret = portfolio_performance(weights, mean_returns, cov_matrix)
    return -((p_ret - risk_free_rate) / p_vol)

def max_sharpe_ratio(mean_returns, cov_matrix, risk_free_rate):
    num_assets = len(mean_returns)
    args = (mean_returns, cov_matrix, risk_free_rate)
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    bound = (0.0,1.0)
    bounds = tuple(bound for asset in range(num_assets))
    result = sco.minimize(neg_sharpe_ratio, num_assets*[1/num_assets], args=args,
                        method='SLSQP', bounds=bounds, constraints=constraints)
    return result

def portfolio_volatility(weights, mean_returns, cov_matrix):
    return portfolio_performance(weights, mean_returns, cov_matrix)[0]

def min_volatility(mean_returns, cov_matrix):
    num_assets = len(mean_returns)
    args = (mean_returns, cov_matrix)
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    bound = (0.0,1.0)
    bounds = tuple(bound for asset in range(num_assets))
    result = sco.minimize(portfolio_volatility, num_assets*[1/num_assets], args=args,
                        method='SLSQP', bounds=bounds, constraints=constraints)
    return result

def neg_treynor_ratio(weights, mean_returns, cov_matrix, risk_free_rate):
    p_vol, p_ret = portfolio_performance(weights, mean_returns, cov_matrix)
    return -(p_ret - risk_free_rate) / beta

def max_treynor_ratio(mean_returns, cov_matrix, risk_free_rate):
    num_assets = len(mean_returns)
    args = (mean_returns, cov_matrix, risk_free_rate)
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    bound = (0.0,1.0)
    bounds = tuple(bound for asset in range(num_assets))
    result = sco.minimize(neg_treynor_ratio, num_assets*[1/num_assets], args=args,
                        method='SLSQP', bounds=bounds, constraints=constraints)
    return result


def risk_contribution(weights, mean_returns, cov_matrix):
    p_variance = portfolio_volatility(weights, mean_returns, cov_matrix)**2
    marginal_contrib = cov_matrix@weights
    risk_contrib = np.multiply(marginal_contrib, weights.T)/p_variance
    return risk_contrib

def target_risk_contributions(target_risk, cov_matrix):
    n = cov_matrix.shape[0]
    init_guess = np.repeat(1/n, n)
    args = (target_risk, cov_matrix)
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    bound = (0.0,1.0)
    bounds = tuple(bound for asset in range(n))
    
    def tsd_risk(weights, target_risk, cov_matrix):
        w_contribs = risk_contribution(weights, mean_returns, cov_matrix)
        return ((w_contribs-target_risk)**2).sum()
    
    weights = sco.minimize(tsd_risk, init_guess, args=args,
                           method='SLSQP', bounds=bounds, constraints=constraints)
    risk_parity_allocation = pd.DataFrame(weights.x,index=returns.columns,columns=['allocation'])
    risk_parity_allocation.allocation = [round(i*100,4)for i in risk_parity_allocation.allocation]
    risk_parity_allocation = risk_parity_allocation.T
    return risk_parity_allocation

def equal_risk_contributions(cov_matrix):
    n =cov_matrix.shape[0]
    return target_risk_contributions(target_risk=np.repeat(1/n, n), cov_matrix=cov_matrix)

def optimized_output(mean_returns, cov_matrix, simulations, risk_free_rate):
    results, _ = random_portfolios(simulations,mean_returns, cov_matrix, risk_free_rate)
    
    max_sharpe = max_sharpe_ratio(mean_returns, cov_matrix, risk_free_rate)
    sdp, rp = portfolio_performance(max_sharpe['x'], mean_returns, cov_matrix)
    max_sharpe_allocation = pd.DataFrame(max_sharpe.x,index=returns.columns,columns=['allocation'])
    max_sharpe_allocation.allocation = [round(i*100,4)for i in max_sharpe_allocation.allocation]
    max_sharpe_allocation = max_sharpe_allocation.T

    min_vol = min_volatility(mean_returns, cov_matrix)
    sdp_min, rp_min = portfolio_performance(min_vol['x'], mean_returns, cov_matrix)
    min_vol_allocation = pd.DataFrame(min_vol.x,index=returns.columns,columns=['allocation'])
    min_vol_allocation.allocation = [round(i*100,4)for i in min_vol_allocation.allocation]
    min_vol_allocation = min_vol_allocation.T
    
    max_treynor = max_treynor_ratio(mean_returns, cov_matrix, risk_free_rate)
    sdp2, rp2 = portfolio_performance(max_treynor['x'], mean_returns, cov_matrix)
    max_treynor_allocation = pd.DataFrame(max_treynor.x,index=returns.columns,columns=['allocation'])
    max_treynor_allocation.allocation = [round(i*100,4)for i in max_treynor_allocation.allocation]
    max_treynor_allocation = max_treynor_allocation.T
    
    equal_risk_allocation = equal_risk_contributions(cov_matrix)
    
    print ("-"*80)
    print ("Risk-free Rate used (%):\n", round(risk_free_rate*100,4))
    print ("-"*80)
    print ("Maximum Sharpe Ratio Portfolio Allocation\n")
    print ("Monthly Return:", round(rp*100,4), "%")
    print ("Monthly Volatility:", round(sdp,4))
    print ("\n")
    print (max_sharpe_allocation)
    print ("-"*80)
    print ("Minimum Volatility Portfolio Allocation\n")
    print ("Monthly Return:", round(rp_min*100,4), "%")
    print ("Monthly Volatility:", round(sdp_min,4))
    print ("\n")
    print (min_vol_allocation)
    print ("-"*80)
    print ("Maximum Treynor Ratio Portfolio Allocation\n")
    print ("Monthly Return:", round(rp2*100,4), "%")
    print ("Beta:", round(beta,4))
    print ("\n")
    print (max_treynor_allocation)
    print ("-"*80)
    print ("Equal Risk-weighted (Parity) Portfolio Allocation\n")
    print ("Monthly Return:", round(rp_min*100,4), "%")
    print ("Monthly Volatility:", round(sdp_min,4))
    print ("\n")
    print (equal_risk_allocation)
    print ("-"*80)

optimized_output(mean_returns, cov_matrix, simulations, risk_free_rate)

--------------------------------------------------------------------------------


C:\Users\pnish\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


<class 'statsmodels.iolib.summary.Summary'>
"""
                             OLS Regression Results                            
===============================================================================
Dep. Variable:     Eq-weight Portfolio   R-squared:                       0.342
Model:                             OLS   Adj. R-squared:                  0.335
Method:                  Least Squares   F-statistic:                     48.43
Date:                 Sun, 15 Mar 2020   Prob (F-statistic):           4.74e-10
Time:                         15:56:38   Log-Likelihood:                 200.34
No. Observations:                   95   AIC:                            -396.7
Df Residuals:                       93   BIC:                            -391.6
Df Model:                            1                                         
Covariance Type:             nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                    0.0029      0.003      0.950      0.345      -0.003       0.009
GSCI Commodity Index     0.4008      0.058      6.959      0.000       0.286       0.515
==============================================================================
Omnibus:                       10.869   Durbin-Watson:                   1.849
Prob(Omnibus):                  0.004   Jarque-Bera (JB):               14.781
Skew:                           0.538   Prob(JB):                     0.000617
Kurtosis:                       4.606   Cond. No.                         18.9
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

--------------------------------------------------------------------------------


<class 'statsmodels.iolib.summary.Summary'>
"""
                             OLS Regression Results                            
===============================================================================
Dep. Variable:     Eq-weight Portfolio   R-squared:                       0.638
Model:                             OLS   Adj. R-squared:                  0.622
Method:                  Least Squares   F-statistic:                     39.72
Date:                 Sun, 15 Mar 2020   Prob (F-statistic):           3.93e-19
Time:                         15:56:38   Log-Likelihood:                 228.74
No. Observations:                   95   AIC:                            -447.5
Df Residuals:                       90   BIC:                            -434.7
Df Model:                            4                                         
Covariance Type:             nonrobust                                         
==============================================================================================
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
const                          0.0051      0.018      0.287      0.775      -0.030       0.040
Fed Rate                       0.0026      0.003      0.880      0.381      -0.003       0.009
Continuous Commodity Index     0.9415      0.093     10.144      0.000       0.757       1.126
US Dollar Index Fund (UUP)    -0.0382      0.148     -0.258      0.797      -0.333       0.256
Treasury Yield 30 Years       -0.0293      0.574     -0.051      0.959      -1.169       1.111
==============================================================================
Omnibus:                       22.189   Durbin-Watson:                   1.826
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               36.789
Skew:                           0.975   Prob(JB):                     1.03e-08
Kurtosis:                       5.344   Cond. No.                         345.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
Risk-free Rate used (%):
 Treasury Yield 30 Years    0.2425
dtype: float64
--------------------------------------------------------------------------------
Maximum Sharpe Ratio Portfolio Allocation

Monthly Return: 0.5771 %
Monthly Volatility: 0.0519


            Agro_product_Fund  Precious_metal_Fund  Industrial_metal_Fund  \
allocation                0.0                  0.0                    0.0   

            Gold_Fund  GenAgro_Fund  Farm_Fund  Energy_Fund  
allocation        0.0           0.0      100.0          0.0  
--------------------------------------------------------------------------------
Minimum Volatility Portfolio Allocation

Monthly Return: -0.1488 %
Monthly Volatility: 0.0277


            Agro_product_Fund  Precious_metal_Fund  Industrial_metal_Fund  \
allocation             1.2283              15.9437  